In [3]:
# test if data is ready
import os

data_dir = "/mnt/sml/data" # replace with your own data dir
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/mnt/sml/data/Data_CHF_Zhao_2020_ATE.csv
/mnt/sml/data/playground-series-s3e15.zip
/mnt/sml/data/playground-series-s3e15/data.csv
/mnt/sml/data/playground-series-s3e15/sample_submission.csv


In [4]:
from pathlib import Path

data_path_0 = Path(data_dir) / "playground-series-s3e15/data.csv"

data_path_1 = Path(data_dir) / "Data_CHF_Zhao_2020_ATE.csv"

In [5]:
# test the main package availability
import torch

print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

a = torch.rand(1000, 1000).to(device)

print(a + a)

True
tensor([[1.4639, 1.3385, 0.1161,  ..., 0.8116, 1.4519, 0.0942],
        [1.6444, 1.9167, 1.4019,  ..., 0.4343, 0.1008, 1.9435],
        [0.3798, 1.9805, 1.2851,  ..., 0.1757, 0.2750, 1.6101],
        ...,
        [0.6892, 1.8756, 1.5452,  ..., 1.3135, 1.0134, 0.2087],
        [0.9109, 0.2480, 0.9178,  ..., 1.1915, 1.7494, 0.7329],
        [1.6215, 1.0548, 0.0330,  ..., 1.4665, 1.7130, 1.6107]],
       device='cuda:0')


In [6]:
# data processing
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_0 = pd.read_csv(data_path_0)
df_1 = pd.read_csv(data_path_1)

print(df_0.head())
print(df_1.head())

   id    author geometry  pressure [MPa]  mass_flux [kg/m2-s]  x_e_out [-]  \
0   0  Thompson     tube            7.00               3770.0       0.1754   
1   1  Thompson     tube             NaN               6049.0      -0.0416   
2   2  Thompson      NaN           13.79               2034.0       0.0335   
3   3      Beus  annulus           13.79               3679.0      -0.0279   
4   4       NaN     tube           13.79                686.0          NaN   

   D_e [mm]  D_h [mm]  length [mm]  chf_exp [MW/m2]  
0       NaN      10.8        432.0              3.6  
1      10.3      10.3        762.0              6.2  
2       7.7       7.7        457.0              2.5  
3       5.6      15.2       2134.0              3.0  
4      11.1      11.1        457.0              2.8  
   id   author geometry  pressure [MPa]  mass_flux [kg/m2-s]  x_e_out [-]  \
0   1  Inasaka     tube            0.39                 5600      -0.1041   
1   2  Inasaka     tube            0.31              

In [7]:
df_0["original"] = 0
df_1["original"] = 1

df_concat = pd.concat([df_0, df_1], axis=0)

df_concat

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],original
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6,0
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2,0
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5,0
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0,0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8,0
...,...,...,...,...,...,...,...,...,...,...,...
1860,1861,Richenderfer,plate,1.01,1500.0,-0.0218,15.0,120.0,10.0,9.4,1
1861,1862,Richenderfer,plate,1.01,1500.0,-0.0434,15.0,120.0,10.0,10.4,1
1862,1863,Richenderfer,plate,1.01,2000.0,-0.0109,15.0,120.0,10.0,10.8,1
1863,1864,Richenderfer,plate,1.01,2000.0,-0.0218,15.0,120.0,10.0,10.9,1


In [8]:
# Transform the text data entry to numerical data
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_concat["original"] = le.fit_transform(df_concat["original"])

df_concat["author"]=le.fit_transform(df_concat["author"])
df_concat["geometry"]=le.fit_transform(df_concat["geometry"])
df_concat["author"]=df_concat["author"].replace({10:np.nan})
df_concat["geometry"]=df_concat["geometry"].replace({2:np.nan})

df_concat


,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],original
0,0,7.0,NaN,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6,0
1,1,7.0,NaN,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2,0
2,2,7.0,3.0,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5,0
3,3,0.0,0.0,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0,0
4,4,NaN,NaN,13.79,686.0,NaN,11.1,11.1,457.0,2.8,0
...,...,...,...,...,...,...,...,...,...,...,...
1860,1861,6.0,1.0,1.01,1500.0,-0.0218,15.0,120.0,10.0,9.4,1
1861,1862,6.0,1.0,1.01,1500.0,-0.0434,15.0,120.0,10.0,10.4,1
1862,1863,6.0,1.0,1.01,2000.0,-0.0109,15.0,120.0,10.0,10.8,1
1863,1864,6.0,1.0,1.01,2000.0,-0.0218,15.0,120.0,10.0,10.9,1


In [9]:
df_clean = df_concat.drop(columns=["x_e_out [-]", "id"])

df_clean

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],original
0,7.0,NaN,7.00,3770.0,NaN,10.8,432.0,3.6,0
1,7.0,NaN,NaN,6049.0,10.3,10.3,762.0,6.2,0
2,7.0,3.0,13.79,2034.0,7.7,7.7,457.0,2.5,0
3,0.0,0.0,13.79,3679.0,5.6,15.2,2134.0,3.0,0
4,NaN,NaN,13.79,686.0,11.1,11.1,457.0,2.8,0
...,...,...,...,...,...,...,...,...,...
1860,6.0,1.0,1.01,1500.0,15.0,120.0,10.0,9.4,1
1861,6.0,1.0,1.01,1500.0,15.0,120.0,10.0,10.4,1
1862,6.0,1.0,1.01,2000.0,15.0,120.0,10.0,10.8,1
1863,6.0,1.0,1.01,2000.0,15.0,120.0,10.0,10.9,1


In [10]:
# Use KNN to impute the missing values, other methods can be used later for robustness testing
from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputer = KNNImputer(n_neighbors=4)

imputer.fit(df_clean)

X_imputed = imputer.transform(df_clean)

df_imputed = pd.DataFrame(X_imputed, columns=df_clean.columns)

df_imputed

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],original
0,7.0,3.0,7.0000,3770.0,10.8,10.8,432.0,3.6,0.0
1,7.0,3.0,9.4775,6049.0,10.3,10.3,762.0,6.2,0.0
2,7.0,3.0,13.7900,2034.0,7.7,7.7,457.0,2.5,0.0
3,0.0,0.0,13.7900,3679.0,5.6,15.2,2134.0,3.0,0.0
4,8.0,3.0,13.7900,686.0,11.1,11.1,457.0,2.8,0.0
...,...,...,...,...,...,...,...,...,...
33504,6.0,1.0,1.0100,1500.0,15.0,120.0,10.0,9.4,1.0
33505,6.0,1.0,1.0100,1500.0,15.0,120.0,10.0,10.4,1.0
33506,6.0,1.0,1.0100,2000.0,15.0,120.0,10.0,10.8,1.0
33507,6.0,1.0,1.0100,2000.0,15.0,120.0,10.0,10.9,1.0


In [12]:
df_imputed.isna().sum() # check if there are any missing values


author                 0
geometry               0
pressure [MPa]         0
mass_flux [kg/m2-s]    0
D_e [mm]               0
D_h [mm]               0
length [mm]            0
chf_exp [MW/m2]        0
original               0
dtype: int64

In [17]:
y = df_concat["x_e_out [-]"]

y = pd.DataFrame(y).reset_index()

y.reset_index()

df_new = pd.concat([df_imputed, y], axis=1)

df_new = df_new.drop(columns=["index"])

df_new

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],original,x_e_out [-]
0,7.0,3.0,7.0000,3770.0,10.8,10.8,432.0,3.6,0.0,0.1754
1,7.0,3.0,9.4775,6049.0,10.3,10.3,762.0,6.2,0.0,-0.0416
2,7.0,3.0,13.7900,2034.0,7.7,7.7,457.0,2.5,0.0,0.0335
3,0.0,0.0,13.7900,3679.0,5.6,15.2,2134.0,3.0,0.0,-0.0279
4,8.0,3.0,13.7900,686.0,11.1,11.1,457.0,2.8,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
33504,6.0,1.0,1.0100,1500.0,15.0,120.0,10.0,9.4,1.0,-0.0218
33505,6.0,1.0,1.0100,1500.0,15.0,120.0,10.0,10.4,1.0,-0.0434
33506,6.0,1.0,1.0100,2000.0,15.0,120.0,10.0,10.8,1.0,-0.0109
33507,6.0,1.0,1.0100,2000.0,15.0,120.0,10.0,10.9,1.0,-0.0218


In [19]:
null_rows = df_new[df_new["x_e_out [-]"].isnull()]

print(null_rows)


       author  geometry  pressure [MPa]  mass_flux [kg/m2-s]  D_e [mm]  \
4        8.00       3.0          13.790               686.00      11.1   
7        5.00       3.0          18.000               750.00      10.0   
10       7.00       3.0          15.945              3510.25       1.9   
12       7.00       3.0           6.890              7500.00      11.8   
23       0.00       0.0          15.510              1355.00       5.6   
...       ...       ...             ...                  ...       ...   
31633    7.00       3.0          11.030              3411.00      11.5   
31634    6.00       1.0           1.010              2000.00      15.0   
31637    8.00       3.0          13.790               688.00      11.1   
31640    7.25       3.0          13.790              3258.25       4.7   
31642    7.00       3.0           6.890              3825.00      23.6   

       D_h [mm]  length [mm]  chf_exp [MW/m2]  original  x_e_out [-]  
4          11.1       457.00            

In [20]:
not_null_rows = df_new[df_new["x_e_out [-]"].notnull()]

print(not_null_rows)

       author  geometry  pressure [MPa]  mass_flux [kg/m2-s]  D_e [mm]  \
0         7.0       3.0          7.0000               3770.0      10.8   
1         7.0       3.0          9.4775               6049.0      10.3   
2         7.0       3.0         13.7900               2034.0       7.7   
3         0.0       0.0         13.7900               3679.0       5.6   
5         7.0       3.0         17.2400               3648.0       1.9   
...       ...       ...             ...                  ...       ...   
33504     6.0       1.0          1.0100               1500.0      15.0   
33505     6.0       1.0          1.0100               1500.0      15.0   
33506     6.0       1.0          1.0100               2000.0      15.0   
33507     6.0       1.0          1.0100               2000.0      15.0   
33508     6.0       1.0          1.0100               2000.0      15.0   

       D_h [mm]  length [mm]  chf_exp [MW/m2]  original  x_e_out [-]  
0          10.8        432.0            

In [21]:
not_null_rows=not_null_rows[not_null_rows["original"]==0]
y=not_null_rows["x_e_out [-]"]
X=not_null_rows.drop(columns=["x_e_out [-]","original","author","geometry"])
X.columns = [col.replace('[', '').replace(']', '').replace('<', '').replace('>', '') for col in X.columns]

In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge, SGDRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
def regress_models(x, X, y):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []
    for train_index, test_index in kf.split(X,y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        x.fit(X_train, y_train)
        y_pred = x.predict(X_test)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    model_name = x.__class__.__name__
    return model_name, round(np.mean(rmse_scores), 3)

algo = [DecisionTreeRegressor(),
        RandomForestRegressor(),
        GradientBoostingRegressor(),
        BaggingRegressor(),
        AdaBoostRegressor(),
        ExtraTreesRegressor(),
        MLPRegressor(),
        KNeighborsRegressor(),
        LinearRegression(),
        CatBoostRegressor(),
        XGBRegressor(),
        LGBMRegressor(),
        # tpot.fitted_pipeline_,
        SGDRegressor(alpha=0.0001, max_iter=1000, tol=1e-3),
        RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=5),
        Ridge(alpha=1.0)]

score = []
for a in algo:
    score.append(regress_models(a, X, y))

pd.DataFrame(score, columns=['Model', 'RMSE'])

Learning rate set to 0.06405
0:	learn: 0.0987641	total: 50.2ms	remaining: 50.2s
1:	learn: 0.0968511	total: 52.1ms	remaining: 26s
2:	learn: 0.0951320	total: 53.4ms	remaining: 17.8s
3:	learn: 0.0935470	total: 63.6ms	remaining: 15.8s
4:	learn: 0.0921715	total: 67.6ms	remaining: 13.5s
5:	learn: 0.0908080	total: 70.2ms	remaining: 11.6s
6:	learn: 0.0896989	total: 72.8ms	remaining: 10.3s
7:	learn: 0.0886968	total: 75.4ms	remaining: 9.35s
8:	learn: 0.0877426	total: 78.3ms	remaining: 8.62s
9:	learn: 0.0868491	total: 79.9ms	remaining: 7.91s
10:	learn: 0.0860493	total: 81.5ms	remaining: 7.33s
11:	learn: 0.0852889	total: 83.3ms	remaining: 6.86s
12:	learn: 0.0846518	total: 85.2ms	remaining: 6.47s
13:	learn: 0.0840017	total: 87.4ms	remaining: 6.16s
14:	learn: 0.0834339	total: 89.8ms	remaining: 5.9s
15:	learn: 0.0828893	total: 99.2ms	remaining: 6.1s
16:	learn: 0.0824122	total: 101ms	remaining: 5.85s
17:	learn: 0.0819943	total: 103ms	remaining: 5.63s
18:	learn: 0.0816245	total: 105ms	remaining: 5.43s


,Model,RMSE
0,DecisionTreeRegressor,1.030000e-01
1,RandomForestRegressor,8.000000e-02
2,GradientBoostingRegressor,7.600000e-02
3,BaggingRegressor,8.300000e-02
4,AdaBoostRegressor,9.400000e-02
5,ExtraTreesRegressor,8.600000e-02
6,MLPRegressor,4.860000e-01
7,KNeighborsRegressor,8.800000e-02
8,LinearRegression,8.600000e-02
9,CatBoostRegressor,7.400000e-02


In [24]:
cat_params = {
    'objective':'RMSE',
    "loss_function": "RMSE",
    'eval_metric': "RMSE",
    "n_estimators": 2000,
    "verbose" : 1000,
    "random_state": 45,
   
}

In [26]:
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np

n_splits = 10
rmse_scores = []
best_iterations = []

for fold, (train_index, val_index) in enumerate(KFold(n_splits=n_splits).split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    model = CatBoostRegressor(**cat_params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val),early_stopping_rounds=500)
    
    best_iteration = model.get_best_iteration()
    best_iterations.append(best_iteration)
    
    y_pred = model.predict(X_val)
    # score = mean_squared_error(y_val, y_pred, squared=False)
    score = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(score)
    print(f"Fold {fold}: Validation RMSE = {score}")
    
mean_rmse_score = np.mean(rmse_scores)
mean_best_iteration = int(np.mean(best_iterations))
print(f"Mean RMSE = {mean_rmse_score}")
print(f"Mean Best Iteration = {mean_best_iteration}")

Learning rate set to 0.053017
0:	learn: 0.0993357	test: 0.0973718	best: 0.0973718 (0)	total: 2.08ms	remaining: 4.15s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.07448465434
bestIteration = 307

Shrink model to first 308 iterations.
Fold 0: Validation RMSE = 0.07448465427489458
Learning rate set to 0.053017
0:	learn: 0.0993843	test: 0.0971794	best: 0.0971794 (0)	total: 1.59ms	remaining: 3.18s
1000:	learn: 0.0673604	test: 0.0712026	best: 0.0710664 (563)	total: 1.86s	remaining: 1.86s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.07106640598
bestIteration = 563

Shrink model to first 564 iterations.
Fold 1: Validation RMSE = 0.07106640598692295
Learning rate set to 0.053017
0:	learn: 0.0994752	test: 0.0960561	best: 0.0960561 (0)	total: 1.98ms	remaining: 3.96s
1000:	learn: 0.0672181	test: 0.0713234	best: 0.0713086 (873)	total: 1.75s	remaining: 1.75s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.07129762585
bestIteration 

In [27]:
cat_params = {
    'objective':'RMSE',
    "loss_function": "RMSE",
    'eval_metric': "RMSE",
    "n_estimators": 757,
    "verbose" : 1000,
    "random_state": 45,
}
model=CatBoostRegressor(**cat_params)
test=null_rows.drop(columns=["x_e_out [-]","original","author","geometry"])
                    
test.columns = [col.replace('[', '').replace(']', '').replace('<', '').replace('>', '') for col in test.columns]
model.fit(X,y)

y_pred=model.predict(test)
y_pred

Learning rate set to 0.083201
0:	learn: 0.0981723	total: 2.1ms	remaining: 1.59s
756:	learn: 0.0666338	total: 1.45s	remaining: 0us


array([-0.00167805, -0.09810541, -0.04546543, ...,  0.03125262,
       -0.04998521, -0.00758705])

In [29]:
sub = pd.DataFrame(y_pred, columns=['x_e_out [-]'])
sub.to_csv('outputs/demo.csv')
sub


,x_e_out [-]
0,-0.001678
1,-0.098105
2,-0.045465
3,0.009570
4,0.037485
...,...
10410,0.077563
10411,-0.047493
10412,0.031253
10413,-0.049985
